In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib

Using matplotlib backend: MacOSX


# Birdsong triangulation
Quick analysis based on time of arrival methods. Suppose a signal arrives at microphones positioned at $A$, $B$ at times $t_a, t_b$. Then the source lies at a distance $2a = (t_a - t_b) u$ further from $A$ than $B$, where $u$ is the speed of sound. If $A, B$ are separated by $2c$ then, introducing Cartesian coordinates we can position $A$ at $(-c, 0)$ and $B$ at $(c, 0)$. The locus of possible origin points of the source is then given by the hyperbola

\begin{equation*}
  \frac{x^2}{a^2} - \frac{y^2}{b^2} = 1.
\end{equation*}

where $b^2 = c^2 - a^2$. We have that $c \ge \lvert a \rvert$ by the geometry of the setup. The branch of hyperbola is determined by the the sign of $a$, and this is encoded nicely in the parametric form of the equations

\begin{align*}
  x &= a \cosh t \\
  y &= b \sinh t
\end{align*}

Note $b > 0$ whereas $a$ is unrestricted in sign.

This is readily generalised, suppose $A, B$ have vector positions $\mathbf{u}, \mathbf{v}$. Then, with $a$ defined as above, and $c = \frac{1}{2} \left\|\mathbf{v} - \mathbf{u}\right\|$, and $b = \sqrt{c^2 - a^2}$

\begin{equation*}
  \mathbf{x}(t) = \frac{1}{2}(\mathbf{u} + \mathbf{v})
  + \frac{a}{2c} \left(\mathbf{v} - \mathbf{u}\right) \cosh t 
  + \frac{b}{2c} \left(\mathbf{v} - \mathbf{u}\right)^\perp \sinh t
\end{equation*}

where $\perp$ denotes a counterclockwise rotation by $90^\circ$.

In principle, with multiple pairs of microphones, the location of the source can be found by finding the intersection of the hyperbolae from the various pairs.

# Solving the linear (asymptote) problem
The asymptotes to each hyperbola, expressed in an appropriate coordinate system are

\begin{equation*}
  y = \pm \frac{b}{a} x
\end{equation*}

We need a method to detect the six asymptotes which approximately line up. Assuming we have these, each asymptote can be written in the generic form

\begin{equation}
  a_i x + b_i y + c_i = 0
\end{equation}
 
where we can specify that $a_i^2 + b_i^2 = 1$. Then, the distance from this asymptote to a point $(x, y)$ in the plane is simply $a_i x + b_i y + c_i$. Working in homogeneous coordinates, we thus seek a point minimising the sum of the squared distances from a point to each of the lines. This can be specified as a constrained least squares problem:

\begin{align*}
  \min \left\| Ax \right\|^2 \\
  \text{s.t } Cx = 1
\end{align*}

where

\begin{align*}
  A &= \begin{bmatrix}
    a_1 & b_1 & c_1 \\
    a_2 & b_2 & c_2 \\
    \vdots & \vdots & \vdots \\
    a_6 & b_6 & c_6
  \end{bmatrix} &
  C &= \begin{bmatrix} 
    0 & 0 & 1
  \end{bmatrix}
\end{align*}

This is an equality-constrained quadratic program, and can be solved by solving the KKT conditions directly:

\begin{equation*}
  \begin{bmatrix}
    2A^TA & C^T \\
    C & 0
  \end{bmatrix}
  \begin{bmatrix}
    x^* \\ y^* \\ z^* \\ \nu^*
  \end{bmatrix} =
  \begin{bmatrix}
    0 \\ 0 \\ 0 \\ 1
  \end{bmatrix}
\end{equation*}

# Problem setup

In [8]:
c = 0.1 #m
c_sound = 330 #m/s

sample_rate = 48000;
Ts = 1/sample_rate;
print(Ts*c_sound)
def time_error():
    return (np.random.rand() - 0.5)*Ts


microphones = np.array([[c, 0], [0, c], [-c, 0], [0, -c]])
source_position =  np.array([75, 1])
arrival_times = np.zeros(4)

for i in range(4):
    arrival_times[i] = (np.linalg.norm(source_position - microphones[i])/c_sound + 
                        time_error())

for i in range(4):
    plt.plot(microphones[i][0], microphones[i][1], 'ko')
plt.plot(source_position[0], source_position[1], 'bo')

0.006875


In [9]:
def asymptote_transform(e1, centre, m):
    x0, y0 = centre
    c, s = e1
    l = np.array([m*c + s, m*s - c, -(m*c+s)*x0 - (m*s - c)*y0])
    l = 1/np.linalg.norm(l[:2])*l
    return l

def compute_y(l, x):
    a, b, c = l
    y = -a/b*x - c/b
    return y
    
def source_locus(u, v, distance_separation):
    # distance_separation = how much further source is from u than v (can be negative)
    a = distance_separation / 2
    
    w = v - u
    c = 0.5*np.linalg.norm(w)
    print(c**2 - a**2)
    if c**2 - a**2 < 0:
        b = 0
    else:
        b = np.sqrt(c**2 - a**2)
    
    e1 = w / (2*c) # axis aligned from u to v
    e2 = np.array([-e1[1], e1[0]]) # perpendicular axis (right-handed)
    
    x0 = 0.5*(u + v) # origin is the midpoint of the two microphones
    
    # Hyperbola
    def hyperbola(t):
        x = x0[0] + a*e1[0]*np.cosh(t) + b*e2[0]*np.sinh(t)
        y = x0[1] + a*e1[1]*np.cosh(t) + b*e2[1]*np.sinh(t)
        return x, y
        
    # Compute asymptote lines in homogeneous form [a, b, c] where a**2 + b**2 = 1
    l_1 = asymptote_transform(e1, x0, b/a)
    l_2 = asymptote_transform(e1, x0, -b/a)
    
    return hyperbola, l_1, l_2

In [10]:
pairs = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
x = np.linspace(-160, 160)
t = np.linspace(-8.5, 8.5, 1000)
L = []
for pair in pairs:
    i1, i2 = pair
    u = microphones[i1]
    v = microphones[i2]
    dist = c_sound*(arrival_times[i1] - arrival_times[i2])
    f_hyp, l_1, l_2 = source_locus(u, v, dist)
    xh, yh = f_hyp(t)
    plt.plot(xh, yh)
    plt.plot(x, compute_y(l_1, x), ':')
    plt.plot(x, compute_y(l_1, x), ':')
    L.append(l_1)
    L.append(l_2)
L = np.array(L)

0.00256076451993
-0.000140954015157
0.00234107458198
0.00236690552504
0.0099952643614
0.00258550267705


In [11]:
match_indices = np.where(sum((abs(np.dot(L[:,:2], L[:,:2].T)) > 0.95).astype('int')) >=4 )
#print(match_indices)
A = L[match_indices]
C = np.array([[0, 0, 1]])
K = np.bmat([[2*np.dot(A.T, A), C.T], [C, [[0]]]])
q = np.array([0, 0, 0, 1])
xsoln = np.linalg.solve(K, q)
plt.plot(xsoln[0], xsoln[1], 'ro')

print(np.linalg.norm(xsoln[:2] - source_position))

75.0064134466


In [ ]:
np.random.rand()